In [2]:
bacteria_list = [
    "Alcaligenes_aquatilis",
    "Aeromonas_australiensis",
    "Campylobacter_jejuni",
    "Caulobacter_flavus",
    "Cyanobacterium_aponinum",
    "Bacillus_cereus",
    "Bacillus_pseudofirmus",
    "Acidobacterium_capsulatum",
    "Leptospira_interrogans",
    "BRC1_bacterium_SCGC_AAA252-M09",
    "Actinomyces_bovis",
    "Gemmatimonas_aurantiaca",
    "Bacteroides_fragilis",
    "Chloroflexus_aurantiacus",
    "Thermotoga_maritima",
    "Aquifex_aeolicus",
    "Gemmata_obscuriglobus",
    "Planctomycetes_bacterium_SCGC_AAA282-C19"
]
archaea_list = [
    "Halovivax_asiaticus",
    "Methanobrevibacter_ruminantium",
    "Methanococcus_aeolicus",
    "Methanococcus_maripaludis",
    "Methanococcus_vannielii",
    "Methanopyrus_kandleri",
    "Thermococcus_gammatolerans",
    "Acidianus_brierleyi",
    "Sulfolobus_acidocaldarius",
    "Staphylothermus_marinus",
    "Thermofilum_pendens",
    "Pyrobaculum_aerophilum",
    "Pyrobaculum_arsenaticum"
]